In [125]:
# unzip input file first

import xml.etree.ElementTree as ET
import requests
import math
import pprint
import time
import numpy as np
import pandas as pd
import os
import collections

## Travis County Boundary API

In [94]:
root = ET.parse('input-files/RoughlyTravisCounty.osm').getroot()

In [95]:
latitudes = []
longitudes = []
for type_tag in root.findall("node"):
    d = type_tag.attrib
    latitudes.append(float(d["lat"]))
    longitudes.append(float(d["lon"]))

## Region Mapping

In [96]:
LATITUDE_MILES = 69
LONGITUDE_MILES = 54.6

vertical_distance = (max(latitudes) - min(latitudes))*LATITUDE_MILES
horizontal_distance = (max(longitudes) - min(longitudes))*LONGITUDE_MILES

In [97]:
# Assuming we want to make 400 squares, how should we partition the vertically/horizontally
NUM_REGIONS = 3200
square_len = (vertical_distance*horizontal_distance/NUM_REGIONS)**0.5

In [122]:
latitude_delta = square_len/LATITUDE_MILES
LATITUDE_DELTA = square_len/LATITUDE_MILES
longitude_delta = square_len/LONGITUDE_MILES
LONGITUDE_DELTA = square_len/LONGITUDE_MILES
longitude_delta

0.013847870403828138

In [99]:
lat_lon = zip(latitudes, longitudes)

In [100]:
min_lat = min(latitudes)
min_lon = min(longitudes)
max_lat = max(latitudes)
max_lon = max(longitudes)
nrows = math.ceil((max_lat - min_lat) / latitude_delta)
ncols = math.ceil((max_lon - min_lon) / longitude_delta)
def map_point_to_region(latitude, longitude):
    return math.floor((latitude-min_lat)/latitude_delta) * ncols  + math.floor((longitude-min_lon)/longitude_delta)

In [101]:
def get_representative(region_num): # 8 cols per row, # R10 --> row_num = 1, col_num = 2
    row_num = region_num//ncols # Correct
    col_num = region_num - row_num*ncols
    lat = min_lat + row_num * latitude_delta + 0.5*latitude_delta
    lon = min_lon + col_num * longitude_delta + 0.5*longitude_delta
    return [lon, lat]

In [102]:
map_point_to_region(30.084446, -97.702885)

36

In [103]:
get_representative(13)

[-98.02650084954831, 30.08076594002934]

In [104]:
valid_regions = set()
for lat, lon in zip(latitudes, longitudes):
    valid_regions.add(map_point_to_region(lat, lon))

In [105]:
valid_regions = list(valid_regions)
valid_regions.sort()

In [106]:
region_coords = [get_representative(i) for i in range(3200)]

[[-98.20652316479809, 30.08076594002934],
 [-98.19267529439426, 30.08076594002934],
 [-98.17882742399043, 30.08076594002934],
 [-98.1649795535866, 30.08076594002934],
 [-98.15113168318277, 30.08076594002934],
 [-98.13728381277895, 30.08076594002934],
 [-98.12343594237511, 30.08076594002934],
 [-98.10958807197129, 30.08076594002934],
 [-98.09574020156747, 30.08076594002934],
 [-98.08189233116363, 30.08076594002934],
 [-98.06804446075981, 30.08076594002934],
 [-98.05419659035597, 30.08076594002934],
 [-98.04034871995215, 30.08076594002934],
 [-98.02650084954831, 30.08076594002934],
 [-98.01265297914449, 30.08076594002934],
 [-97.99880510874067, 30.08076594002934],
 [-97.98495723833683, 30.08076594002934],
 [-97.97110936793301, 30.08076594002934],
 [-97.95726149752917, 30.08076594002934],
 [-97.94341362712535, 30.08076594002934],
 [-97.92956575672152, 30.08076594002934],
 [-97.9157178863177, 30.08076594002934],
 [-97.90187001591387, 30.08076594002934],
 [-97.88802214551004, 30.08076594002

 [-97.44489029258754, 30.343755061437694],
 [-97.43104242218371, 30.343755061437694],
 [-97.41719455177989, 30.343755061437694],
 [-97.40334668137605, 30.343755061437694],
 [-97.38949881097223, 30.343755061437694],
 [-97.37565094056839, 30.343755061437694],
 [-98.20652316479809, 30.354712941496377],
 [-98.19267529439426, 30.354712941496377],
 [-98.17882742399043, 30.354712941496377],
 [-98.1649795535866, 30.354712941496377],
 [-98.15113168318277, 30.354712941496377],
 [-98.13728381277895, 30.354712941496377],
 [-98.12343594237511, 30.354712941496377],
 [-98.10958807197129, 30.354712941496377],
 [-98.09574020156747, 30.354712941496377],
 [-98.08189233116363, 30.354712941496377],
 [-98.06804446075981, 30.354712941496377],
 [-98.05419659035597, 30.354712941496377],
 [-98.04034871995215, 30.354712941496377],
 [-98.02650084954831, 30.354712941496377],
 [-98.01265297914449, 30.354712941496377],
 [-97.99880510874067, 30.354712941496377],
 [-97.98495723833683, 30.354712941496377],
 [-97.971109

 [-97.95726149752917, 30.53003902243528],
 [-97.94341362712535, 30.53003902243528],
 [-97.92956575672152, 30.53003902243528],
 [-97.9157178863177, 30.53003902243528],
 [-97.90187001591387, 30.53003902243528],
 [-97.88802214551004, 30.53003902243528],
 [-97.87417427510621, 30.53003902243528],
 [-97.86032640470238, 30.53003902243528],
 [-97.84647853429856, 30.53003902243528],
 [-97.83263066389472, 30.53003902243528],
 [-97.8187827934909, 30.53003902243528],
 [-97.80493492308707, 30.53003902243528],
 [-97.79108705268324, 30.53003902243528],
 [-97.77723918227942, 30.53003902243528],
 [-97.76339131187558, 30.53003902243528],
 [-97.74954344147176, 30.53003902243528],
 [-97.73569557106792, 30.53003902243528],
 [-97.7218477006641, 30.53003902243528],
 [-97.70799983026028, 30.53003902243528],
 [-97.69415195985644, 30.53003902243528],
 [-97.68030408945262, 30.53003902243528],
 [-97.66645621904878, 30.53003902243528],
 [-97.65260834864496, 30.53003902243528],
 [-97.63876047824112, 30.530039022435

## Batch calls of distance matrix

In [107]:
# Prepare data for api call

locations = region_coords
# locations = region_coords + ems_coords + hospital_coords
durations = [[0 for i in range(3200)] for j in range(3200)]

#pprint.pprint(locations)
print(len(locations))

3200


In [109]:
batch_size = 1
num_batches = math.ceil(len(valid_regions) / batch_size)
num_batches_per_round = math.ceil(num_batches/4)
current_branch = 3
remainder = len(valid_regions) % batch_size
print(num_batches)

1948


In [110]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf624884518d9d4075408e95e3401165976247',
#     'Authorization': '5b3ce3597851110001cf62485eef3a0ab19d4b3e8ad2d8c88dca3d19',
    'Content-Type': 'application/json; charset=utf-8'
}

In [111]:
for i in range(num_batches_per_round*current_branch, min(num_batches_per_round*current_branch+num_batches_per_round, len(valid_regions))):
    print("batch ", i)
    start = i * batch_size
    sources = list(range(start, start + batch_size))

    # handle last partial batch
    if(sources[batch_size - 1] >= len(locations)):
        sources = sources[:remainder]

    locations = [get_representative(valid_regions[j]) for j in range(len(valid_regions))]

    body = {"locations": locations, "sources": sources}
    call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

    duration_batch = call.json()['durations']

    for j in range(len(duration_batch)):
        for k in range(len(duration_batch[0])):
            durations[valid_regions[i*batch_size + j]][valid_regions[k]] = duration_batch[j][k]
    time.sleep(0.1)


batch  1461
batch  1462
batch  1463
batch  1464
batch  1465
batch  1466
batch  1467
batch  1468
batch  1469
batch  1470
batch  1471
batch  1472
batch  1473
batch  1474
batch  1475
batch  1476
batch  1477
batch  1478
batch  1479
batch  1480
batch  1481
batch  1482
batch  1483
batch  1484
batch  1485
batch  1486
batch  1487
batch  1488
batch  1489
batch  1490
batch  1491
batch  1492
batch  1493
batch  1494
batch  1495
batch  1496
batch  1497
batch  1498
batch  1499
batch  1500
batch  1501
batch  1502
batch  1503
batch  1504
batch  1505
batch  1506
batch  1507
batch  1508
batch  1509
batch  1510
batch  1511
batch  1512
batch  1513
batch  1514
batch  1515
batch  1516
batch  1517
batch  1518
batch  1519
batch  1520
batch  1521
batch  1522
batch  1523
batch  1524
batch  1525
batch  1526
batch  1527
batch  1528
batch  1529
batch  1530
batch  1531
batch  1532
batch  1533
batch  1534
batch  1535
batch  1536
batch  1537
batch  1538
batch  1539
batch  1540
batch  1541
batch  1542
batch  1543
batc

In [43]:
len(valid_regions)/490

3.975510204081633

In [30]:
print(len(durations), len(durations[0]))

467 467


In [112]:
np.save("durations4.npy", durations)

In [108]:
durations = np.load("durations4.npy", allow_pickle=True)

In [186]:
# Read all census tracts and put points in dict
CENSUS_TRACTS_DIR = "census_tracts"
census_tracts_points = {}
for fname in os.listdir(CENSUS_TRACTS_DIR):
    latitudes = []
    longitudes = []
    try:
        # Parse the XML file
        root = ET.parse("{}/{}".format(CENSUS_TRACTS_DIR, fname)).getroot()
        for type_tag in root.findall("node"):
            d = type_tag.attrib
            latitudes.append(float(d["lat"]))
            longitudes.append(float(d["lon"]))
        census_tracts_points[fname.strip(".txt")] = list(zip(latitudes, longitudes))
        census_tracts_bounding[fname.strip(".txt")] = [(min(latitudes), min(longitudes)), (max(latitudes), max(longitudes))]
    except:
        print(fname)

18_04.txt
17_69.txt
17_41.txt
17_55.txt
22_11.txt
17_82.txt
17_83.txt
22_10.txt
17_54.txt
17_40.txt
17_68.txt
18_11.txt
18_05.txt
18_39.txt
3_02.txt
20_03.txt
16_02.txt
18_13.txt
17_56.txt
17_42.txt
22_12.txt
17_81.txt
17_80.txt
22_07.txt
17_57.txt
9800.txt
18_06.txt
18_12.txt
16_03.txt
20_02.txt
3_05.txt
14_02.txt
17_53.txt
17_47.txt
17_84.txt
22_02.txt
17_85.txt
1_01.txt
17_46.txt
17_52.txt
14_03.txt
12.txt
16_06.txt
18_17.txt
3_04.txt
3_06.txt
20_05.txt
18_29.txt
16_04.txt
10.txt
13_08.txt
14_01.txt
17_50.txt
17_78.txt
22_01.txt
17_86.txt
17_79.txt
17_51.txt
17_45.txt
1_02.txt
11.txt
16_05.txt
18_28.txt
20_04.txt
3_07.txt
9_01.txt
24_07.txt
24_13.txt
17_22.txt
17_37.txt
24_12.txt
9_02.txt
18_58.txt
18_64.txt
24_10.txt
23_19.txt
21_08.txt
19_18.txt
19_19.txt
21_09.txt
24_11.txt
23_18.txt
18_59.txt
18_61.txt
18_49.txt
24_29.txt
23_08.txt
17_18.txt
17_19.txt
19_08.txt
24_28.txt
18_48.txt
18_60.txt
18_62.txt
24_02.txt
17_33.txt
24_03.txt
18_63.txt
18_46.txt
24_26.txt
24_32.txt
23_13.txt

In [187]:
def in_grid(grid_lat, grid_lon, lat, lon):
    return (grid_lat <= lat <= grid_lat + LATITUDE_DELTA and grid_lon <= lon <= grid_lon + LONGITUDE_DELTA)

In [188]:
census_tract_grids = collections.defaultdict(set)
visited = set()
for tract_name in census_tracts_points:
    for lat, lon in census_tracts_points[tract_name]:
        region = map_point_to_region(lat, lon)
#         if region not in visited:
        census_tract_grids[tract_name].add(region)
#         visited.add(region)
    census_tract_grids[tract_name] = list(census_tract_grids[tract_name])

In [189]:
dict(census_tract_grids)

{'18_04': [1440, 1561, 1499, 1500, 1501, 1438, 1439],
 '17_69': [997, 998, 935, 936, 937, 938, 875, 876, 877, 878, 939],
 '17_41': [1665,
  1666,
  1971,
  1972,
  1973,
  1849,
  1850,
  1851,
  1852,
  1726,
  1727,
  1728,
  1729,
  1730,
  2033,
  1910,
  1911,
  1912,
  1788,
  1789,
  1790,
  1791],
 '17_55': [1921,
  1922,
  1923,
  1860,
  1861,
  1862,
  1799,
  1800,
  1801,
  1739,
  1740,
  1982,
  1983],
 '22_11': [1018,
  1200,
  1201,
  1202,
  1139,
  1140,
  1141,
  1142,
  1079,
  1080,
  1081,
  1078,
  1019],
 '17_82': [1920, 1921, 2042, 2043, 2044, 1981, 1982, 1983],
 '17_83': [1304,
  1549,
  1550,
  1487,
  1488,
  1489,
  1490,
  1427,
  1428,
  1429,
  1366,
  1367,
  1368,
  1426,
  1306,
  1365,
  1551],
 '22_10': [1152,
  1153,
  1025,
  1024,
  1028,
  1029,
  1030,
  903,
  904,
  905,
  906,
  907,
  1147,
  1206,
  1145,
  783,
  1022,
  1151,
  1450,
  1026,
  1332,
  1327,
  1027,
  1329,
  1330,
  1203,
  1204,
  1205,
  1331,
  1207,
  1208,
  1209,


In [190]:
import json

d = {"latitude_min": min_lat, "longitude_min": min_lon, "latitude_max": max_lat, "longitude_max": max_lon, "latitude_step": latitude_delta, "longitude_step": longitude_delta}
d["time_matrix"] = durations.tolist()
d["census_tract_region_mapping"] = dict(census_tract_grids)
with open("grid_info_multiple.json", "w") as j:
    json.dump(d, j)

In [155]:
2045 in valid_regions

True

In [158]:
l = list(census_tract_grids.values())

In [164]:
l2 = list(np.concatenate(l).ravel())

In [167]:
2046 in l2

True

In [178]:
census_tract_grids["23_07"]

[887, 888, 826, 827, 765, 766]